In [58]:
import pandas as pd
import numpy as np

lots = pd.read_csv('../clean-files/artsy_auctions_artworks_info.csv')
lots.columns

Index(['url', 'img_url', 'Price', 'Artist', 'Artist_url', 'Title',
       'Pre-sale estimate', 'Medium', 'Sale Date', 'Auction house',
       'Price_USD', 'Height (cm)', 'Width (cm)', 'Area (cm²)', 'Price_fix',
       'Price_unit', 'Sale details', 'Price (USD)', 'Price (USD / cm²)',
       'Price (USD / cm)', 'Technique', 'Sold', 'Year', 'Year of sale'],
      dtype='object')

In [59]:
prints = lots[lots['Technique'] == 'print']
paintings = lots[lots['Technique'] == 'painting']
drawings = lots[lots['Technique'] == 'drawing']

print('Prints: ', len(prints))
print('Paintings: ', len(paintings))
print('Drawings: ', len(drawings))

Prints:  6080
Paintings:  3032
Drawings:  263


In [60]:
# Median price of each medium
prints_median_price = prints['Price (USD)'].median()
paintings_median_price = paintings['Price (USD)'].median()
drawings_median_price = drawings['Price (USD)'].median()
# Weight of each medium
prints_weight = prints_median_price / (prints_median_price + paintings_median_price + drawings_median_price)
paintings_weight = paintings_median_price / (prints_median_price + paintings_median_price + drawings_median_price)
drawings_weight = drawings_median_price / (prints_median_price + paintings_median_price + drawings_median_price)

print('Prints weight: ', prints_weight.round(2))
print('Paintings weight: ', paintings_weight.round(2))
print('Drawings weight: ', drawings_weight.round(2))

Prints weight:  0.06
Paintings weight:  0.82
Drawings weight:  0.12


In [61]:
artists_names = lots['Artist'].unique()

artists_indices = pd.DataFrame()

for artist_name in artists_names:
    print(artist_name)
    artist_lots = lots[lots['Artist'] == artist_name]

    # ignore top and bottom 5% of lots by 'Price (USD / cm)'
    percentiles = artist_lots['Price (USD / cm)'].quantile([0.05, 0.95])
    artist_lots = artist_lots[(artist_lots['Price (USD / cm)'] > percentiles[0.05]) & (artist_lots['Price (USD / cm)'] < percentiles[0.95])]
    artist_lots = artist_lots[artist_lots['Price (USD / cm)'] > 0]

    # artist dataframe: index: years (from first to last lot), columns: prints_median_price, paintings_median_price, drawings_median_price
    artist_df = pd.DataFrame(columns=['prints_median_price', 'paintings_median_price', 'drawings_median_price'])
    artist_df['prints_median_price'] = prints[prints['Artist'] == artist_name].groupby('Year of sale')['Price (USD)'].median()
    artist_df['paintings_median_price'] = paintings[paintings['Artist'] == artist_name].groupby('Year of sale')['Price (USD)'].median()
    artist_df['drawings_median_price'] = drawings[drawings['Artist'] == artist_name].groupby('Year of sale')['Price (USD)'].median()

    artist_df = artist_df.fillna(0)

    artist_df['prints_weight'] = np.where(artist_df['prints_median_price'] == 0, 0, prints_weight)
    artist_df['paintings_weight'] = np.where(artist_df['paintings_median_price'] == 0, 0, paintings_weight)
    artist_df['drawings_weight'] = np.where(artist_df['drawings_median_price'] == 0, 0, drawings_weight)

    artist_df['total_weight'] = artist_df['prints_weight'] + artist_df['paintings_weight'] + artist_df['drawings_weight']

    artist_df['Price Index'] = (
        artist_df['prints_weight'] * artist_df['prints_median_price'] +
        artist_df['paintings_weight'] * artist_df['paintings_median_price'] +
        artist_df['drawings_weight'] * artist_df['drawings_median_price']
    ) / artist_df['total_weight']

    # apply 3-year moving average
    artist_df['Price Index'] = artist_df['Price Index'].rolling(window=3).mean()

    # Assign the relevant portion of 'Price Index' to the corresponding column in 'artists_indices'
    artists_indices[artist_name] = artist_df['Price Index']

artists_indices.round(2)

Marc Chagall
Emiliano Di Cavalcanti
Vicente do Rego Monteiro
Cândido Portinari
Victor Vasarely


,Marc Chagall,Emiliano Di Cavalcanti,Vicente do Rego Monteiro,Cândido Portinari,Victor Vasarely
Year of sale,,,,,
1992,NaN,NaN,NaN,NaN,NaN
1993,NaN,NaN,NaN,NaN,NaN
1995,313402.53,NaN,NaN,NaN,NaN
1996,269031.86,NaN,NaN,NaN,NaN
1997,247098.21,NaN,NaN,NaN,NaN
1998,205958.47,NaN,NaN,NaN,NaN
1999,224050.02,NaN,NaN,NaN,NaN
2000,187095.00,NaN,NaN,NaN,NaN
2001,243581.04,NaN,NaN,NaN,NaN


In [62]:
# year of sale as index
artists_indices.to_csv('./models/artsy_auctions_artists_indices.csv')